In [7]:
import json
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
import pymesh

In [2]:
ds_path = Path('/ws/data/sds/itodd')
models_path = ds_path / 'models'

In [3]:
dir_path = ds_path / 'data_debug/000000'
fname = '000002.hdf5'
fpath = dir_path / fname
f = h5py.File(fpath.as_posix(), 'r')

In [4]:
f.keys()

<KeysViewHDF5 ['blender_proc_version', 'colors', 'gt', 'segmap', 'segmap_key_to_num']>

In [5]:
f['colors'], f['gt'], f['segmap'], f['segmap_key_to_num']

(<HDF5 dataset "colors": shape (512, 512, 3), type "|u1">,
 <HDF5 dataset "gt": shape (), type "|S16679">,
 <HDF5 dataset "segmap": shape (512, 512), type "|u1">,
 <HDF5 dataset "segmap_key_to_num": shape (), type "|S268">)

In [6]:
segmap = f['segmap'][...]
segmap.shape, segmap.dtype, np.unique(segmap)


((512, 512),
 dtype('uint8'),
 array([ 0,  2,  7, 16, 19, 25, 49, 65, 66, 70, 72], dtype=uint8))

In [14]:
amap_str = f['segmap_key_to_num'][...].item().decode('utf-8')
amap_str

'{"itodd_obj_000002_01": 2, "itodd_obj_000006_01": 6, "itodd_obj_000008_01": 8, "itodd_obj_000009_01": 9, "itodd_obj_000014_01": 14, "itodd_obj_000015_01": 15, "itodd_obj_000016_01": 16, "itodd_obj_000023_01": 23, "itodd_obj_000028_01": 28, "tless_obj_000018_01": 46, "tless_obj_000020_01": 48, "tless_obj_000022_01": 50, "itodd_obj_000021_02": 65, "itodd_obj_000016_02": 66, "itodd_obj_000021_03": 68, "itodd_obj_000028_02": 69, "itodd_obj_000007_03": 71, "itodd_obj_000018_02": 72, "itodd_obj_000002_03": 76}'

In [15]:
amap = json.loads(amap_str)
amap

{'itodd_obj_000002_01': 2,
 'itodd_obj_000006_01': 6,
 'itodd_obj_000008_01': 8,
 'itodd_obj_000009_01': 9,
 'itodd_obj_000014_01': 14,
 'itodd_obj_000015_01': 15,
 'itodd_obj_000016_01': 16,
 'itodd_obj_000023_01': 23,
 'itodd_obj_000028_01': 28,
 'tless_obj_000018_01': 46,
 'tless_obj_000020_01': 48,
 'tless_obj_000022_01': 50,
 'itodd_obj_000021_02': 65,
 'itodd_obj_000016_02': 66,
 'itodd_obj_000021_03': 68,
 'itodd_obj_000028_02': 69,
 'itodd_obj_000007_03': 71,
 'itodd_obj_000018_02': 72,
 'itodd_obj_000002_03': 76}

In [14]:
img = f['colors'][...]
img.shape, img.dtype, type(img)

((512, 512, 3), dtype('uint8'), numpy.ndarray)

In [28]:
gt = f['gt'][...]
gt.shape, gt.dtype, type(gt)

((), dtype('S24807'), numpy.ndarray)

In [29]:
gt_str = f['gt'][...].item().decode('utf-8')
gt = json.loads(gt_str)

In [31]:
len(gt['objects'])

60

In [18]:
attr_map = f['instance_attribute_maps'][...]
attr_map.shape, attr_map.dtype

((), dtype('S732'))

In [19]:
with open('temp.json', 'w') as jf:
    json.dump(gt, jf, indent=2)

In [19]:
obj_fpath = models_path / 'obj_000001.ply'
mesh = pymesh.load_mesh(obj_fpath.as_posix())

In [20]:
mesh.get_attribute_names()

['face_vertex_indices', 'vertex_x', 'vertex_y', 'vertex_z']

In [21]:
mesh.add_attribute('vertex_normal')

In [22]:
normals = mesh.get_attribute('vertex_normal').reshape((-1, 3))
normals.shape, normals.dtype, np.allclose(np.linalg.norm(normals, axis=1), 1)

((18613, 3), dtype('float64'), True)

In [13]:
mesh.get_attribute('vertex_x').shape

(18613,)

In [17]:
normals = mesh.get_vertex_attribute('vertex_normal')
normals.shape, normals.dtype, np.allclose(np.linalg.norm(normals, axis=1), 1)

((18613, 3), dtype('float64'), True)

In [18]:
if mesh.has_attribute('vertex_normal'):
    mesh.remove_attribute('vertex_normal')
mesh.get_attribute_names()

['face_area',
 'face_normal',
 'face_vertex_indices',
 'vertex_x',
 'vertex_y',
 'vertex_z']

In [24]:
pymesh.save_mesh('test.ply', mesh, 'vertex_normal', ascii=True)

In [13]:
data = [[256, 6], [128, 8], [64, 16], [32, 32], [16, 64], [8, 128], [4, 256], [2, 512], [1, 1024]]
data = np.concatenate([np.arange(len(data)).reshape((-1, 1)), data], axis=1)
df = pd.DataFrame(data, index='step', columns=('step', 'size', 'channels'))
df

TypeError: Index(...) must be called with a collection of some kind, 'step' was passed